<a href="https://colab.research.google.com/github/tgandor/urban_oculus/blob/master/results/Validate_multiple_JPEG_0mod4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
import os
if not os.path.isdir('urban_oculus'):
  !git clone https://github.com/tgandor/urban_oculus
# first, this doesn't kill runtime
%run urban_oculus/snippets/get_coco_val2017_nounzip.ipynb
# this may kill runtime
%run urban_oculus/snippets/get_detectron2.ipynb

CPU times: user 17.9 ms, sys: 6.79 ms, total: 24.7 ms
Wall time: 1.64 s
/usr/bin/mogrify
CPU times: user 192 ms, sys: 35.3 ms, total: 227 ms
Wall time: 797 ms
CPU times: user 345 ms, sys: 67.5 ms, total: 413 ms
Wall time: 16.2 s


In [0]:
import os
import shutil

def filesizes(glob_expr):
  import glob
  return {
      os.path.basename(f): os.path.getsize(f)
      for f in glob.glob(glob_expr)
  }


def validate_coco2017(quality, output_dir=None):
  def save(path):
    if output_dir:
      os.makedirs(output_dir, exist_ok=True)
      shutil.copy(path, output_dir)
      print(f'Saving {path} to {output_dir}')

  import json
  import time
  print('unpacking')
  !unzip -o val2017.zip -d datasets/coco 2>&1 | tqdm --total 5000 >/dev/null
  !du -h datasets/coco/val2017
  with open(f'filesizes_original.json', 'w') as f:
    f.write(json.dumps({
        'quality': 'original',
        'filesizes': filesizes('datasets/coco/val2017/*.jpg'),
        'elapsed': 0,
    }))
  save('filesizes_original.json')  

  print(f'degrading quality to {quality}')
  start = time.time()
  !mogrify -verbose -quality {quality} datasets/coco/val2017/*.jpg 2>&1 | tqdm --total 10000 >/dev/null
  elapsed = time.time() - start
  !du -h datasets/coco/val2017

  import detectron2.model_zoo
  from detectron2.config import get_cfg
  from detectron2.data import build_detection_test_loader
  from detectron2.evaluation import COCOEvaluator, inference_on_dataset
  from detectron2.engine import DefaultPredictor
  cfg = get_cfg()
  cfg.merge_from_file(detectron2.model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
  cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
  predictor = DefaultPredictor(cfg)
  data_loader = build_detection_test_loader(cfg, 'coco_2017_val')
  evaluator = COCOEvaluator('coco_2017_val', cfg, False)
  start = time.time()
  results = inference_on_dataset(predictor.model, data_loader, evaluator)
  inference_time = time.time() - start

  result_file = f'coco_val2017_rcnn_3x_q_{quality}.json'
  with open(result_file, 'w') as f:
    f.write(json.dumps({
        'quality': quality,
        'bbox': results['bbox'],
        'elapsed': inference_time,
    }))
  save(result_file)
  
  sizes_file = f'filesizes_{quality}.json'
  with open(sizes_file, 'w') as f:
    f.write(json.dumps({
        'quality': quality,
        'filesizes': filesizes('datasets/coco/val2017/*.jpg'),
        'elapsed': elapsed,
    }))
  save(sizes_file)

  # running locally
  from IPython.core.display import display, HTML
  display(HTML(f'Results: <a href="{result_file}" target="_blank">{result_file}</a> (not on Colab)'))

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls /content/gdrive/My\ Drive/results

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
20200220_112237_results.tar.gz	20200220_162619_results


In [4]:
%%time
import time
output_dir = None
if os.path.exists('/content/gdrive'):
  output_dir = f'/content/gdrive/My Drive/results/{time.strftime("%Y%m%d_%H%M%S")}_results'
  print('Using output dir:', output_dir)

# this should go into some snippet:
try:
  from tqdm.notebook import tqdm
except ImportError:
  from tqdm import tqdm_notebook as tqdm

for q in tqdm(range(0, 101, 4)):
  validate_coco2017(q, output_dir)

Using output dir: /content/gdrive/My Drive/results/20200221_090234_results


unpacking
5002it [00:05, 912.25it/s]              
788M	datasets/coco/val2017
Saving filesizes_original.json to /content/gdrive/My Drive/results/20200221_090234_results
degrading quality to 0
100% 10000/10000 [01:05<00:00, 152.82it/s]
625M	datasets/coco/val2017


model_final_f10217.pkl: 178MB [00:06, 25.6MB/s]                           


Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=16.11s).
Accumulating evaluation results...
DONE (t=2.12s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.371
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.542
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.415
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.401
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.503
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.300
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.433
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.441
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

unpacking
5001it [00:05, 896.70it/s]              
788M	datasets/coco/val2017
Saving filesizes_original.json to /content/gdrive/My Drive/results/20200221_090234_results
degrading quality to 4
100% 10000/10000 [00:43<00:00, 229.83it/s]
51M	datasets/coco/val2017
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=7.67s).
Accumulating evaluation results...
DONE (t=1.41s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.016
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.029
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.016
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.004
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.015
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.030
 Average Recall     (AR) @[ IoU=0.50:0.95 | are

...

unpacking
5001it [00:05, 869.67it/s]              
788M	datasets/coco/val2017
Saving filesizes_original.json to /content/gdrive/My Drive/results/20200221_090234_results
degrading quality to 88
100% 10000/10000 [01:03<00:00, 157.91it/s]
504M	datasets/coco/val2017
Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=16.02s).
Accumulating evaluation results...
DONE (t=2.22s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.370
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.540
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.412
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.194
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.399
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.504
 Average Recall     (AR) @[ IoU=0.50:0.95 | 

unpacking
5001it [00:06, 823.21it/s]              
788M	datasets/coco/val2017
Saving filesizes_original.json to /content/gdrive/My Drive/results/20200221_090234_results
degrading quality to 92
100% 10000/10000 [01:08<00:00, 145.50it/s]
625M	datasets/coco/val2017
Loading and preparing results...
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=15.96s).
Accumulating evaluation results...
DONE (t=2.24s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.371
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.542
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.415
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.401
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.503
 Average Recall     (AR) @[ IoU=0.50:0.95 | 

unpacking
5001it [00:06, 829.59it/s]              
788M	datasets/coco/val2017
Saving filesizes_original.json to /content/gdrive/My Drive/results/20200221_090234_results
degrading quality to 96
100% 10000/10000 [01:13<00:00, 136.94it/s]
826M	datasets/coco/val2017
Loading and preparing results...
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=15.80s).
Accumulating evaluation results...
DONE (t=2.21s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.374
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.545
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.419
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.199
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.404
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.508
 Average Recall     (AR) @[ IoU=0.50:0.95 | 

unpacking
5001it [00:05, 841.81it/s]              
788M	datasets/coco/val2017
Saving filesizes_original.json to /content/gdrive/My Drive/results/20200221_090234_results
degrading quality to 100
100% 10000/10000 [01:22<00:00, 121.07it/s]
1.3G	datasets/coco/val2017
Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=15.62s).
Accumulating evaluation results...
DONE (t=2.19s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.375
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.546
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.419
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.199
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.404
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.509
 Average Recall     (AR) @[ IoU=0.50:0.95 |


CPU times: user 2h 32min 22s, sys: 39min 59s, total: 3h 12min 21s
Wall time: 3h 59min 30s


In [0]:
import time
now = time.strftime('%Y%m%d_%H%M%S')
all_jsons = f"{now}_results.tar.gz"
# adding 'surrounding' folder around JSONs (avoiding tarbomb)
!tar --xform 's+^+{now}_results/+' -czf {all_jsons} *.json
!ls -l

In [0]:
import os
if os.path.exists('/content/gdrive'):
  !mkdir -p /content/gdrive/My\ Drive/results
  !cp -v {all_jsons} /content/gdrive/My\ Drive/results/
  !ls /content/gdrive/My\ Drive/results/

In [0]:
try:
  from google.colab import files
  files.download(all_jsons)
except ImportError:
  # running locally
  from IPython.core.display import display, HTML
  display(HTML(f'Results archive: <a href="{all_jsons}" target="_blank">{all_jsons}</a>'))

In [0]:
import torch

print('Device count:', torch.cuda.device_count())

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3, 1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3, 1), 'GB')


In [0]:
!head -n 11 /proc/cpuinfo

In [5]:
!ls                                                                                        

annotations_trainval2017.zip	 filesizes_100.json
coco_val2017_rcnn_3x_q_0.json	 filesizes_12.json
coco_val2017_rcnn_3x_q_100.json  filesizes_16.json
coco_val2017_rcnn_3x_q_12.json	 filesizes_20.json
coco_val2017_rcnn_3x_q_16.json	 filesizes_24.json
coco_val2017_rcnn_3x_q_20.json	 filesizes_28.json
coco_val2017_rcnn_3x_q_24.json	 filesizes_32.json
coco_val2017_rcnn_3x_q_28.json	 filesizes_36.json
coco_val2017_rcnn_3x_q_32.json	 filesizes_40.json
coco_val2017_rcnn_3x_q_36.json	 filesizes_44.json
coco_val2017_rcnn_3x_q_40.json	 filesizes_48.json
coco_val2017_rcnn_3x_q_44.json	 filesizes_4.json
coco_val2017_rcnn_3x_q_48.json	 filesizes_52.json
coco_val2017_rcnn_3x_q_4.json	 filesizes_56.json
coco_val2017_rcnn_3x_q_52.json	 filesizes_60.json
coco_val2017_rcnn_3x_q_56.json	 filesizes_64.json
coco_val2017_rcnn_3x_q_60.json	 filesizes_68.json
coco_val2017_rcnn_3x_q_64.json	 filesizes_72.json
coco_val2017_rcnn_3x_q_68.json	 filesizes_76.json
coco_val2017_rcnn_3x_q_72.json	 filesizes_80.json
coc